In [ ]:
import numpy as np
import pandas as pd
import scipy as sp

import plotly.offline as py
import plotly.graph_objs as go
import plotly
plotly.offline.init_notebook_mode(connected=True)

import ipywidgets

import datetime
import math
import bisect
import time
from os import listdir
from os.path import isfile, join

In [ ]:
# Takes a list/numpy array of pairs of XY vectors to plot
class XYChart:
    
    def __init__(self, x=None, y=None, x_label='x', y_label='y', title='Plot', names=None):
        self.num_datasets = 0
        
        self.x_label = x_label
        self.y_label = y_label
        self.title = title
        self.names = names
        
        self.chart = go.FigureWidget()
        self.update(x=x, y=y, x_label=x_label, y_label=y_label, title=title, names=names)
        self.display()
    
    def update(self, x=None, y=None, x_label=None, y_label=None, title=None, names=None):
        
        # Don't overwrite these if not specified
        if x_label is None:
            x_label = self.x_label
        if y_label is None:
            y_label = self.y_label
        if title is None:
            title = self.title
        if names is None:
            names = self.names
            
        # Check if y is none
        if y is None:
            y = [np.zeros([1,1])]
        # Format the x and y data into lists of 1D numpy arrays
        (x_format, y_format) = format_xy_data(x, y)
        
        # Pad list of names with y0, y1, etc
        num_datasets = len(y_format)
        names = pad_names(names, num_datasets)
        
        # Clear current data (needed if num datasets goes down to clear extra legend entries, but prevent flashing)
        if self.num_datasets != num_datasets:
            #self.chart.data = self.chart.data[:num_datasets]
            self.chart.data = []
            self.num_datasets = num_datasets
        
        # Make data dict from formatted x and y data
        data = []
        for ii in range(num_datasets):
            data.append(dict(type = 'scatter',
                             x = np.array(x_format[ii]),
                             y = np.array(y_format[ii]),
                             mode='lines',
                             name=names[ii]
                            )
                       )
        
        layout = dict(
                        title = title,

                        xaxis=dict(
                            title=x_label,
                        ),

                        yaxis=dict(
                            title=y_label,
                        )
                     )
        
        #self.chart = go.FigureWidget( data=data, layout=layout )
        self.chart.update(dict( data=data, layout=layout ))
        
    def display(self):
        display(self.chart)
    

In [ ]:
# Check if 1D by checking len() of each element in list/array
def check_if_1d(x):
    # Axis along which plotting vector is stored. Assume plotting is along longest dimension
    plot_axis = 0
    all_elements_same_length = True
    is_1D = True
    last_length = 0
    if x is not None:
        for ii in range(len(x)):
            element = x[ii]
            try:
                length = len(element)
                # If any of the elements are longer than the number of elements, then plot axis is along elements
                if length > len(x):
                    plot_axis = 1
            except TypeError:
                length = 1

            if ii > 0 and last_length != length:
                all_elements_same_length = False
            last_length = length

            if length > 1:
                is_1D = False

        if plot_axis == 0 and all_elements_same_length:
            plot_axis = 0
        else:
            plot_axis = 1
    else:
        is_1D = True
        plot_axis = 0
        
    return (is_1D, plot_axis)

In [ ]:
def format_xy_data(x_in, y_in):
        (y_data_1d, plot_axis) = check_if_1d(y_in)
        x_data_1d = check_if_1d(x_in)[0]
        ### Convert to list of 1D numpy arrays ###
        # 1D
        if y_data_1d:
            y = [np.array(y_in).reshape(len(y_in))]
            if x_in is not None:
                x = [np.array(x_in).reshape(len(x_in))]
            else:
                x = [np.arange(len(y[0]))]
                
        # Plot axis is along columns
        elif plot_axis == 0:
            y = list(np.array(y_in).transpose())
            if x_in is not None:
                if x_data_1d:
                    x = [np.array(x_in).reshape(len(x_in))] * len(y)
                else:
                    x = list(np.array(x_in).transpose())
            else:
                x = [np.arange(len(y[0])) for ii in range(len(y))]
        
        # Plot axis is along rows
        elif plot_axis == 1:
            y = [np.array(e).reshape(len(e)) for e in y_in]
            if x_in is not None:
                if x_data_1d:
                    x = [np.array(x_in).reshape(len(x_in))] * len(y)
                else:
                    x = [np.array(e).reshape(len(e)) for e in x_in]
            else:
                x = [np.arange(len(y[ii])) for ii in range(len(y))]
        return (x, y)

In [ ]:
def pad_names(names, num_datasets):
    if names is not None:
        num_names = len(names)
        names = list(names)
    else:
        num_names = 0
        names = []

    for ii in range(num_datasets - num_names):
        names.append('y' + str(ii))
    return names

In [ ]:
class CandlestickChart:
    
    def updateChart(self, placeholder):
        print(1)
        starttime = datetime.datetime.combine(self.start_date, datetime.time())
        endtime = datetime.datetime.combine(self.end_date, datetime.time()) + datetime.timedelta(days=1)
        if starttime >= endtime:
            return
        
        print(2)
        chartdata = self.fullpricedata[self.fullpricedata['DateTime'] >= starttime]
        chartdata = self.fullpricedata[self.fullpricedata['DateTime'] < endtime]
        
        pdata = self.chart.data[0]
        #pdata.x = [dt.strftime("%y/%m/%d, %H:%M:%S") for dt in chartdata['DateTime']]
        pdata.x = [ii for ii in range(len(chartdata))]
        pdata.low = chartdata['Low']
        pdata.open = chartdata['Open']
        pdata.close = chartdata['Close']
        pdata.high = chartdata['High']
        print(3)
#        layout = self.chart.layout
#         layout.xaxis.tickvals = [ii for ii in range(len(pdata.x)) if 
#                                  chartdata['DateTime'][ii].time() == datetime.time(9, 30)]
#         print(4)
#         xaxisdatetimes = [chartdata['DateTime'][ii] for ii in layout.xaxis.tickvals]
#         layout.xaxis.ticktext = [dt.strftime(' %y/%m/%d ') if dt.time() == datetime.time(9, 30)
#                                  else dt.strftime(' %H:%M ') for dt in xaxisdatetimes]
#        print(5)
    
    def __init__(self, instrument, start_date, end_date):
        self.instrument = instrument
        self.start_date = start_date
        self.end_date = end_date
        
        minutedatadir = r'''C:\ProgramData\Kibot Agent\Data\SP500_1'''
        minutecolnames = ['Date', 'Time', 'Open', 'High', 'Low', 'Close', 'Volume']
        self.fullpricedata = pd.read_csv(join(minutedatadir, instrument+'.txt'), names=minutecolnames)
        self.fullpricedata['DateTime'] = (self.fullpricedata['Date']+self.fullpricedata['Time']).map(lambda x: datetime.datetime(int(x[6:10]), int(x[0:2]), int(x[3:5]), int(x[10:12]), int(x[13:15])))
        
        data = [dict(
                 type = 'candlestick',
                 name = 'Price Data')
               ]
        layout = dict(
                title = 'Candlestick Chart',
                xaxis = dict(
                    type = 'category', 
                    categoryorder = 'category ascending',
                    title = dict(
                        text = 'DateTime'
                    )
                ),
                yaxis = dict(
                    title = dict(
                        text = 'Price'
                    )
                ))
        self.chart = go.FigureWidget( data=data, layout=layout)
        
        # DEFINE DISPLAY CONFIGURATION
        self.display = self.chart
        
        self.updateChart(0)

In [ ]:


# chart_date = datetime.date(2018,1,8)
# candlestick_chart = CandlestickChart('STX', chart_date, chart_date)

In [ ]:
# if __name__ == '__main__':
#     minutedatadir = r'''C:\Users\matth\AlgoTrading\probable-spork\Data\KibotData\SP500_1'''
#     minutecolnames = ['Date', 'Time', 'Open', 'High', 'Low', 'Close', 'Volume']
#     fullpricedata = pd.read_csv(join(minutedatadir, 'ALXN.txt'), names=minutecolnames)
#     fullpricedata['DateTime'] = (fullpricedata['Date']+fullpricedata['Time']).map(lambda x: datetime.datetime(int(x[6:10]), int(x[0:2]), int(x[3:5]), int(x[10:12]), int(x[13:15])))
#     date = datetime.date(2018,3,15)
#     fullpricedata = fullpricedata[[(dt >= datetime.datetime(date.year, date.month, date.day, 9, 30, 0)) and (dt <= datetime.datetime(date.year, date.month, date.day, 16, 0, 0)) for dt in fullpricedata['DateTime']]].copy()
#     xy_chart = XYChart()
#     xy_chart.update(y=fullpricedata['Open'].tolist())